In [1]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras import *
from keras.models import *
from keras.layers import Input, Dense
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input, decode_predictions

import numpy as np
from keras.utils.np_utils import to_categorical
import glob
import os.path
from keras.models import load_model

categories = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

Using TensorFlow backend.


In [2]:
def load_data(file_path):
    x_train = []
    y_train = []
    
    images = glob.glob(file_path + '/**/*')
    for photo in images:
        img = image.load_img(photo, target_size=(299, 299))
        tr_x = image.img_to_array(img)
        tr_x = preprocess_input(tr_x)
        label = (photo.split('\\')[1][2:])
        label_place = categories.index(label)
        
        x_train.append(tr_x)
        y_train.append(label_place)
        
    return np.array(x_train), to_categorical(y_train)

In [3]:
x_train, y_train = load_data('D:/Datasets/scene-classification/data/train/')

In [4]:
type(y_train)

numpy.ndarray

In [5]:
y_train.shape

(15000, 6)

In [8]:
x_train.shape

(15000, 299, 299, 3)

In [9]:
np.save('D:/Datasets/scene-classification/train_set_array.npy', x_train)

In [12]:
input = Input(shape=(299, 299, 3))

In [15]:
if (os.path.isfile('irnv2.h5')):
    print('Model exists')
    model = load_model('irnv2.h5')
else:
    print('Model does not exist, starting training')
    base_model = InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=input, input_shape=(299, 299, 3), pooling='avg', classes=1000)
    for l in base_model.layers:
        l.trainable=False
    
    t = base_model(input)
    o = Dense(len(categories), activation='softmax')(t)
    model = Model(inputs=input, outputs=o)
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
    model.fit(x_train, y_train, batch_size=32, epochs=3, shuffle=True, verbose=1)
    model.save('irnv2.h5')

Model does not exist, starting training
Epoch 1/3
15000/15000 [==============================] - 220s 15ms/step - loss: 0.5428 - categorical_accuracy: 0.8163
Epoch 2/3
15000/15000 [==============================] - 179s 12ms/step - loss: 0.3589 - categorical_accuracy: 0.8701
Epoch 3/3
15000/15000 [==============================] - 192s 13ms/step - loss: 0.3320 - categorical_accuracy: 0.8772
